# Preprocessing

## Check which patients have 5min and 15min bg measurements

In [1]:
import os
import pandas as pd

from src.features.tuners import XGBHyperparameterTuner

In [2]:
train_data_file = os.path.join('..', '..', '..', '..', 'data', 'raw', 'train.csv')
train_data = pd.read_csv(train_data_file, index_col=0, low_memory=False)
patient_ids_train_data = train_data['p_num'].unique()
patient_ids_train_data

array(['p01', 'p02', 'p03', 'p04', 'p05', 'p06', 'p10', 'p11', 'p12'],
      dtype=object)

In [3]:
test_data_file = os.path.join('..', '..', '..', '..', 'data', 'raw', 'test.csv')
test_data = pd.read_csv(test_data_file, index_col=0, low_memory=False)
patient_ids_test_data = test_data['p_num'].unique()
patient_ids_test_data

array(['p01', 'p02', 'p04', 'p05', 'p06', 'p10', 'p11', 'p12', 'p15',
       'p16', 'p18', 'p19', 'p21', 'p22', 'p24'], dtype=object)

In [4]:
all_patient_ids = sorted(list(set(train_data['p_num'].unique().tolist() + test_data['p_num'].unique().tolist())))
all_patient_ids

['p01',
 'p02',
 'p03',
 'p04',
 'p05',
 'p06',
 'p10',
 'p11',
 'p12',
 'p15',
 'p16',
 'p18',
 'p19',
 'p21',
 'p22',
 'p24']

In [5]:
bg_columns = [train_data.columns[i] for i in range(len(train_data.columns)) if 'bg-' in train_data.columns[i]]

columns = ['p_num'] + bg_columns
joined = pd.concat([train_data[columns], test_data[columns]], axis=0)

joined['no_nan_values'] = joined[bg_columns].notna().sum(axis=1)
joined['nan_values'] = joined[bg_columns].isna().sum(axis=1)

# sum up the number of nan and non_nan values for each patient
grouped = joined.groupby('p_num').agg({'no_nan_values': 'sum', 'nan_values': 'sum'})
grouped['total_values'] = grouped['no_nan_values'] + grouped['nan_values']

# make this relative to the total number of values
grouped['rel_nan_values'] = round(grouped['nan_values'] / grouped['total_values'], 2)
grouped['rel_no_nan_values'] = round(grouped['no_nan_values'] / grouped['total_values'], 2)

grouped['in_train_data'] = grouped.index.isin(patient_ids_train_data)
grouped['in_test_data'] = grouped.index.isin(patient_ids_test_data)

grouped = grouped.drop(columns=['no_nan_values', 'nan_values', 'total_values'])

grouped

,rel_nan_values,rel_no_nan_values,in_train_data,in_test_data
p_num,,,,
p01,0.67,0.33,True,True
p02,0.01,0.99,True,True
p03,0.01,0.99,True,False
p04,0.01,0.99,True,True
p05,0.68,0.32,True,True
p06,0.68,0.32,True,True
p10,0.01,0.99,True,True
p11,0.02,0.98,True,True
p12,0.02,0.98,True,True


# Model selection

Based in this table we can see that there are 3 groups of patients:
1. Patients that are only in the train data
2. Patients that are only in the test data
3. Patients that are in both the train and test data

We can use this information to decide which patients we want to train a specific model on and which patients we want to use a generic model for.

Patients that are in both the train and test data will be used to train a specific model on.

All other patients will be used to train a generic model on.

In [6]:
# show the p_num of the patients that are in both the train and test data
patients_with_own_model = list(grouped[(grouped['in_train_data'] == True) & (grouped['in_test_data'] == True)].index)
patients_with_generic_model = list(grouped[(grouped['in_train_data'] == False) & (grouped['in_test_data'] == True)].index)

## Train a specific model for patients that are in both the train and test data

In [ ]:
import joblib
from pipelines import pipeline

for patient_id in patients_with_own_model:
    print(f'-----------{patient_id}-----------')
    data = train_data[train_data['p_num'] == patient_id]
    train_data_patient = pipeline.fit_transform(data)
    X = train_data_patient.drop(columns=['bg+1:00'])
    y = train_data_patient['bg+1:00']
    tuner = XGBHyperparameterTuner(search_space='deep')
    tuner.fit(X=X, y=y)
    
    print('Best hyperparameters found.')
    print(f'RMSE: {tuner.get_rmse()}')
    tuner.show_chart()
    
    joblib.dump(tuner.get_best_model(), f'XGBRegressor.{patient_id}.model.pkl')

-----------p01-----------


In [ ]:
import os
import pandas as pd

train_data_file = os.path.join('..', '..', '..', '..', 'data', 'raw', 'train.csv')
df = pd.read_csv(train_data_file, index_col=0, low_memory=False)
df.head()

In [ ]:
from pipelines import preprocessing_pipeline

df = preprocessing_pipeline.fit_transform(df)
display(df.head())
df.describe()

In [ ]:
from pipelines import standardization_pipeline

df = standardization_pipeline.fit_transform(df)
display(df.head())
df.describe()